In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Any results you write to the current directory are saved as output.

In [2]:
data =  pd.read_csv('../data/train.csv', nrows = 15_000_000)


In [ ]:
# Given a dataframe, add two new features 'abs_diff_longitude' and
# 'abs_diff_latitude' reprensenting the "Manhattan vector" from
# the pickup location to the dropoff location.
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(data)


In [ ]:
print('Old size: %d' % len(data))
data = data.dropna(how = 'any', axis = 'rows')
print('New size: %d' % len(data))

In [ ]:
#print(data.describe())
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a)) # 2*R*asin...

# add new column to dataframe with distance in miles
data['distance_miles'] = distance(data.pickup_latitude, data.pickup_longitude, \
                                      data.dropoff_latitude, data.dropoff_longitude)

(data.head())

In [ ]:
print('Old size: %d' % len(data))
data = data[(data.abs_diff_longitude < 3.0) & (data.abs_diff_latitude < 3.0)]
data = data[(data.pickup_longitude >= -74.3) & (data.pickup_longitude <= -72.9)]  # nyc coordinates
data = data[(data.dropoff_longitude >= -74.3) & (data.dropoff_longitude <= -72.9)]
data = data[(data.pickup_latitude >= 40.5) & (data.pickup_latitude <= 41.8)]
data = data[(data.dropoff_latitude >= 40.5) & (data.dropoff_latitude <= 41.8)]
data = data[(data.fare_amount>=2) & (data.fare_amount<=500)]
data = data[(data.passenger_count>0) & (data.passenger_count <=6)]
data = data[(data.distance_miles<=100.0) & (data.distance_miles>0.05)]
nyc = (-74.0063889, 40.7141667)
data['distance_to_center'] = distance(nyc[1], nyc[0],data.dropoff_latitude, data.dropoff_longitude)
data = data[data.distance_to_center<15.0]
print('New size: %d' % len(data))